# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:

# Configure the map plot
print("Step 1: Displaying the initial map of cities.")
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    color="Humidity",
    tiles="OSM",
    title="City Locations by Humidity"
)
city_map

# Display the map
# YOUR CODE HERE

Step 1: Displaying the initial map of cities.


/opt/anaconda3/lib/python3.12/site-packages/shapely/constructive.py:180: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/opt/anaconda3/lib/python3.12/site-packages/shapely/constructive.py:180: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/opt/anaconda3/lib/python3.12/site-packages/shapely/constructive.py:180: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
print("Step 2: Filtering cities for ideal weather conditions.")
ideal_conditions = city_data_df[
    (city_data_df["Max Temp"] >= 21) & 
    (city_data_df["Max Temp"] <= 27) & 
    (city_data_df["Wind Speed"] < 10) & 
    (city_data_df["Cloudiness"] == 0)
]
ideal_conditions = ideal_conditions.dropna()  
print("Filtered cities:", ideal_conditions)


# Display sample data
ideal_conditions.head()

Step 2: Filtering cities for ideal weather conditions.
Filtered cities:      City_ID                City      Lat       Lng  Max Temp  Humidity  \
9          9        saint-pierre -21.3393   55.4781     22.82        68   
45        45               kapaa  22.0752 -159.3190     22.99        84   
51        51                hilo  19.7297 -155.0900     26.27        83   
63        63               banda  25.4833   80.3333     24.62        52   
73        73               nalut  30.3333   10.8500     25.41        27   
81        81       makakilo city  21.3469 -158.0858     21.66        81   
146      146  puerto del rosario  28.5004  -13.8627     24.86        73   
152      152             kahului  20.8947 -156.4700     23.80        60   
197      197                 gat  31.6100   34.7642     24.38       100   
211      211              laguna  38.4210 -121.4238     21.67        79   
240      240         tikaitnagar  26.9500   81.5833     23.56        59   
258      258  comodoro rivad

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
45,45,kapaa,22.0752,-159.3190,22.99,84,0,3.60,US,1666108257
51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
73,73,nalut,30.3333,10.8500,25.41,27,0,5.11,LY,1666108275


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
print("Step 3: Preparing hotel DataFrame.")
hotel_df = ideal_conditions[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
hotel_df["Hotel Name"] = ""  # Add empty column for hotel names
print("Sample hotel_df:", hotel_df.head())

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

Step 3: Preparing hotel DataFrame.
Sample hotel_df:             City Country      Lat       Lng  Humidity Hotel Name
9   saint-pierre      RE -21.3393   55.4781        68           
45         kapaa      US  22.0752 -159.3190        84           
51          hilo      US  19.7297 -155.0900        83           
63         banda      IN  25.4833   80.3333        52           
73         nalut      LY  30.3333   10.8500        27           


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
print("Step 4: Starting hotel search with Geoapify API.")
base_url = "https://api.geoapify.com/v2/places"
radius = 10000  # Search radius in meters

for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Parameters for the Geoapify API request
    params = {
        "categories": "accommodation.hotel",
        "filter": f"circle:{longitude},{latitude},{radius}",
        "bias": f"proximity:{longitude},{latitude}",
        "limit": 1,
        "apiKey": geoapify_key
    }

    # API request
    response = requests.get(base_url, params=params)
    response_data = response.json()

    # Extract hotel name from the API response
    try:
        hotel_name = response_data["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    print(f"City: {row['City']} - Hotel: {hotel_df.loc[index, 'Hotel Name']}")


# Display sample data
hotel_df

Step 4: Starting hotel search with Geoapify API.
City: saint-pierre - Hotel: Tropic Hotel
City: kapaa - Hotel: Pono Kai Resort
City: hilo - Hotel: Dolphin Bay Hotel
City: banda - Hotel: #acnindiafy21
City: nalut - Hotel: No hotel found
City: makakilo city - Hotel: Embassy Suites by Hilton Oahu Kapolei
City: puerto del rosario - Hotel: Hotel Tamasite
City: kahului - Hotel: Maui Seaside Hotel
City: gat - Hotel: No hotel found
City: laguna - Hotel: Holiday Inn Express & Suites
City: tikaitnagar - Hotel: No hotel found
City: comodoro rivadavia - Hotel: No hotel found
City: san quintin - Hotel: Jardines Hotel
City: santa rosalia - Hotel: Sol y Mar
City: pemba - Hotel: Hotel Rafael
City: narwar - Hotel: No hotel found
City: port hedland - Hotel: The Esplanade Hotel
City: roebourne - Hotel: No hotel found
City: saint-francois - Hotel: Hôtel Epona
City: capoterra - Hotel: Rosa Hotel
City: stolac - Hotel: Bregava
City: guerrero negro - Hotel: Plaza sal paraiso
City: bandarbeyla - Hotel: No hote

,City,Country,Lat,Lng,Humidity,Hotel Name
9,saint-pierre,RE,-21.3393,55.4781,68,Tropic Hotel
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
73,nalut,LY,30.3333,10.8500,27,No hotel found
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
146,puerto del rosario,ES,28.5004,-13.8627,73,Hotel Tamasite
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,Holiday Inn Express & Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:


# Configure the map plot
print("Step 5: Creating a map with hover info for hotels.")
hotel_df["Hover Info"] = (
    "City: " + hotel_df["City"] + 
    ", Country: " + hotel_df["Country"] + 
    ", Hotel: " + hotel_df["Hotel Name"]
)

final_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    color="Humidity",
    tiles="OSM",
    hover_cols=["Hover Info"],
    title="Hotel Locations with Hover Info"
)
final_map

# Display the map
# YOUR CODE HERE

Step 5: Creating a map with hover info for hotels.


/opt/anaconda3/lib/python3.12/site-packages/shapely/constructive.py:180: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/opt/anaconda3/lib/python3.12/site-packages/shapely/constructive.py:180: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/opt/anaconda3/lib/python3.12/site-packages/shapely/constructive.py:180: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hover Info)